# [LVV-T2232] - M1M3 Integration with SAL

The objective of this test case is to verify the latest M1M3 commands, events, and telemetry defined by the latest version of the XML.  
This test case will exercise the functionality of the M1M3 on the 3rd level of the Summit and meets the following criteria:
- Only requires the most current version of SAL
- Only requires the M1M3 surrogate to be loaded on the cell
- Requires the use of the DDS and the EFD

[LVV-T2232]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2232

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by b1quint on 2022-06-14T20:20:59.382.
  Running in yagan07 at summit



---
## LVV-T1996 (1.0) M1M3 DDS Startup Procedure

[LVV-T1996 (1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1996

In [3]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.cRIOpy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

In [4]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [5]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [6]:
domain = salobj.Domain()

In [7]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [8]:
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

In [9]:
index = 22321285  # Test Case + Test Execution

start_time = datetime.now()
script = salobj.Controller("Script", index=index)

In [10]:
await mtcs.set_state(state=salobj.State.DISABLED, components=["mtm1m3"], overrides={"mtm1m3": "Default"})

In [11]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm1m3"])

In [12]:
script.log.info("LVV-T12232 - LVV-E1285 - Start")

---
## Telemetry Verification

Verify the MTM1M3_forceActuatorData telemetry data is being published to the EFD with the following parameters:

- primaryCylinderForce
- secondaryCylinderForce
- xForce
- yForce
- zForce
- fx
- fy
- fz
- mx
- my
- mz
- timestamp
- forceMagnitude

Check [Chronograph - M1M3 Status].

[Chronograph - M1M3 Status]: https://chronograf-tucson-teststand-efd.lsst.codes/sources/1/dashboards/37?refresh=Paused&lower=now%28%29%20-%205m

In [15]:
if exec_info.loc == "summit":
    client = EfdClient("summit_efd")
elif location == "tucson":
    client = EfdClient("tucson_teststand_efd")
else:
    raise ValueError(
        "Location does not match any valid options {summit|tucson}"
    )

In [16]:
start = time.Time("2022-06-14T20:20", scale="utc", format="isot")
end = time.Time("2022-06-14T20:30", scale="utc", format="isot")

In [19]:
df = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [22]:
df

,forceMagnitude,fx,fy,fz,mx,my,mz,primaryCylinderForce0,primaryCylinderForce1,primaryCylinderForce10,...,zForce90,zForce91,zForce92,zForce93,zForce94,zForce95,zForce96,zForce97,zForce98,zForce99
2022-06-14 20:23:30.431000+00:00,3809.418457,85.224068,-1504.604858,-3498.652588,-715.060120,-201.551117,-71.750298,10.187027,-19.931446,-10.469072,...,-37.499535,-14.496274,-59.257969,-74.290695,-22.195852,-51.223316,-11.185081,-54.310261,-43.609318,-23.358471
2022-06-14 20:23:30.443000+00:00,3807.615479,84.975540,-1503.668823,-3497.098145,-705.331177,-204.163101,-73.644165,10.089388,-19.931446,-10.469072,...,-37.555645,-14.585337,-59.141220,-74.337372,-22.070658,-51.223316,-11.249735,-54.287102,-43.534378,-23.335596
2022-06-14 20:23:30.463000+00:00,3811.184814,85.337730,-1504.894043,-3500.448730,-725.897766,-203.434479,-70.273987,10.154481,-20.117722,-10.500606,...,-37.654259,-14.641379,-59.234718,-74.267357,-21.900042,-51.445724,-11.185081,-54.389175,-43.507938,-23.312719
2022-06-14 20:23:30.483000+00:00,3808.606689,84.858192,-1504.350830,-3497.886719,-707.882690,-201.457596,-73.328827,10.187027,-20.055630,-10.532139,...,-37.654259,-14.417210,-59.180103,-74.304642,-22.061573,-51.190529,-11.152755,-54.329121,-43.312222,-23.368305
2022-06-14 20:23:30.503000+00:00,3811.266113,85.243378,-1504.893555,-3500.539795,-709.605957,-202.002640,-70.493622,10.154481,-20.148767,-10.374472,...,-37.644630,-14.506272,-59.187920,-74.290695,-21.941431,-51.435753,-11.217408,-54.268242,-43.344841,-23.335596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-14 20:29:59.908000+00:00,0.517302,-0.003042,0.423209,0.297466,-3.725654,3.883183,2.852129,-0.162732,0.031046,0.000000,...,0.046486,0.109066,-0.093497,0.079405,0.557271,0.188120,-0.032327,0.009429,0.088158,0.022876
2022-06-14 20:29:59.929000+00:00,1.844737,-0.073257,-1.232412,-1.370711,-6.511684,0.679224,1.755750,-0.195278,0.093138,0.031533,...,-0.056113,0.076044,-0.124663,0.074845,0.422991,0.390584,-0.064654,-0.042020,0.045838,0.026081
2022-06-14 20:29:59.949000+00:00,0.405582,0.132096,0.378276,0.062885,-1.638163,5.462717,2.044622,-0.065093,0.031046,0.063067,...,-0.056113,0.030002,-0.077964,0.023339,0.548187,0.357797,0.000000,0.009429,-0.009700,0.098131
2022-06-14 20:29:59.969000+00:00,1.033179,0.129351,-0.939686,-0.409531,-3.692890,3.253726,1.106678,-0.260371,0.062092,0.094600,...,-0.023243,0.033022,-0.046799,-0.102744,0.483574,0.210935,0.096980,0.088343,0.111077,0.078461


In [20]:
df.iloc[0]

forceMagnitude    3809.418457
fx                  85.224068
fy               -1504.604858
fz               -3498.652588
mx                 -715.06012
                     ...     
zForce95           -51.223316
zForce96           -11.185081
zForce97           -54.310261
zForce98           -43.609318
zForce99           -23.358471
Name: 2022-06-14 20:23:30.431000+00:00, Length: 552, dtype: object

---
Verify the MTM1M3_forceActuatorPressure telemetry data is being published to the EFD with the following parameters:
- timestamps
- primaryCylinderPullPressures
- primaryCylinderPushPressures
- secondaryCylinderPullPressures
- secondaryCylinderPushPressures

In [23]:
fap_df = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorPressure", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [24]:
fap_df

""


---
Verify the MTM1M3_inclinometerDatatelemetry data is being published to the EFD with the following parameters:
- timestamp
- inclinometerAngle

In [33]:
df_id = await client.select_time_series(
    "lsst.sal.MTM1M3.outerLoopData", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [34]:
df_id

""


In [ ]:
script.log.info("LVV-T12232 - LVV-E1285 - END")